In [1]:
from ClfTemplates import get_all_template_path, sample_template, sample_template_path
from ClfUse import showClfResult, get_samples, labeledDataFromJson
from Ner_model import get_X_Y, lstm_crf, predict, preprocess
from Ner_data_make import create_fake_label, fake2real
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import re
import numpy as np

Using TensorFlow backend.


### 制造样本 label

通过模版，制造样本

In [2]:
banklist = ['SBIUPI', 'SBIOTP', 'KOTKBK', 'MYACCT', 'BOBTXN',
            'RBISAY', 'CSHBCK', 'ATMSBI', 'UMOBIL', 'PAYTMB']

template = '/data-0/tigergraph/template'
all_templates = get_all_template_path(template_dir = template)
sampled_template_path = sample_template_path(all_templates)
sampled_template_df = sample_template(sampled_template_path, banklist, num_samples=1)

# 全部银行模版中，只取top10打标签；
count_bank = sampled_template_df.groupby('bank').count()
top_10_bank = count_bank.sort_values(by ='sms').head(10).index.tolist()
top_10_sampled_template_df = sampled_template_df[sampled_template_df.bank.isin(top_10_bank)]
top_10_sampled_template_df['unique_template_id'] = top_10_sampled_template_df.bank + '_' + top_10_sampled_template_df.cluster_id

# top10打标签结果；
top_10_train_path = '/home/qibo/项目2_sms/综合/bank-top-10-data/sms_top_10_bank.json'
labeled_df = labeledDataFromJson(top_10_train_path)
labeled_df.loc[labeled_df.label == 'error', 'label'] = ['流水转出确认支付密码']

# 拼接到一起；
top_10_sampled_template_df['label'] = labeled_df.label.tolist()
top_10_sampled_template_df['label'] = top_10_sampled_template_df.label.apply(lambda x: x[0] if isinstance(x, list) else x)



total templates among all the banks:42


In [3]:
####  整合之前top10 bank 打的标签数据
path = 'bank_round1_ner_new.csv'
top10_data = pd.read_csv(path,header=None)
top10_data = top10_data.dropna()
top10_data.columns = ['token', 'lable']

id_list = top_10_sampled_template_df.unique_template_id.tolist()
id_list = [i.lower() for i in id_list]
top10_data['sent_id'] = top10_data.token.apply(lambda x: x if x in id_list else np.nan)
top10_data.sent_id=top10_data.sent_id.fillna(method='ffill')
top10_data.lable = top10_data.lable.apply(lambda x: x if x !='O' else 'other')

In [4]:
## 造假20倍
dupli = 40
df = create_fake_label(top10_data, dupli)
df2 = fake2real(df, dupli)

/home/tigergraph/test1/lib/python3.5/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/tigergraph/test1/lib/python3.5/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/tigergraph/test1/lib/python3.5/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

/home/tigergraph/test1/lib/python3.5/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/tigergraph/test1/lib/python3.5/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/tigergraph/test1/lib/python3.5/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [5]:
df2.shape

(28120, 3)

### 模型优化；

### 主动学习，低熵框架

#### 展示短信分类结果；

In [48]:
sms = get_samples(banklist, sampled_template_path)
showClfResult(sms, top_10_sampled_template_df)

processing sentence num:0
new_instance:Your Loan Approved  Complete Process: Click http://bit.ly/2wCBTha--

此短信来自于模版:CSHBCK_3

元组第三个代表模版ID:(1.0000000000000002, '贷前申请＿审核通过', 'CSHBCK_3')


'贷前申请＿审核通过'

In [7]:
x, y, vocab_size, n_tags, tokenizer, maxlen,label2idx = get_X_Y(df2)
model, history = lstm_crf(x, y, vocab_size, n_tags, epochs = 25, batch_size = 50)

n_tags:4
maxlen:51
vocab_size:3388
label2idx:{'I-date': 0, 'I-amount': 1, 'other': 2, 'I-bal': 3}
X_train:(920, 51)
y_train:(51, 4)
Instructions for updating:
Colocations handled automatically by placer.


/home/tigergraph/generate_template_qb_test/Ner_model.py:20: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  tmp.label = lables.tolist()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/tigergraph/test1/lib/python3.5/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 51)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 51, 30)            101640    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 51, 100)           32400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 51, 50)            5050      
_________________________________________________________________
crf_1 (CRF)                  (None, 51, 4)             228       
Total params: 139,318
Trainable params: 139,318
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8

In [8]:
def get_train_sample():
    import random
    leng = len(ALL)
    idx = random.sample(range(leng), 1)[0]
    return [ALL[idx][0].split()]

def get_test_sample():
    import random
    sms = sample_template(sampled_template_path, banklist, num_samples=30)
    leng = len(sms)
    idx = random.sample(range(leng), 1)[0]
    sampled = sms.iloc[idx,:]
    sample_sms, sample_info = [sampled.sms.split()], sampled.bank + '_' + str(sampled.cluster_id)
    return sample_sms, sample_info

def get_train_sample2(DF):
    import random
    tmp = DF.groupby(['sent_id']).apply(lambda x: x.token.tolist())
    leng = len(tmp)
    idx = random.sample(range(leng), 1)[0]
    return [tmp[idx]]


In [49]:
clf_res(' '.join(new_sms[0]).strip(), top_10_sampled_template_df)

processing sentence num:0
 dear customer you have sent rs. 40. 00 to add-money@paytm from your paytm bank savings account xxxxxxxx9873. upi reference: 822820165975. queries write to us at http://m. p-y. tm/care

元组第三个代表模版ID:(0.7810259251879443, '流水-转出', 'PAYTMB_1')


'流水-转出'

In [120]:
class showFinalResult:
    def __init__(self, sms, model, tokenizer, label2idx, top_10_sampled_template_df):
        '''sms 只能是两种格式：
           格式1： [['this', 'is', 'type', '1']];
           格式2:  DF with sms columns;'''
        self.sms = sms
        self.model = model
        self.tokenizer = tokenizer
        self.label2idx = label2idx
        self.top_10_sampled_template_df = top_10_sampled_template_df
        
    @property
    def get_cls(self):
        cls = showClfResult(' '.join(self.sms[0]).strip(), self.top_10_sampled_template_df)
        return cls
    
    @property
    def get_ner(self):
        lab, tok = self._ner_res(self.sms, self.tokenizer, self.label2idx, self.model)
        ner_dic = self._ner_dic_form(lab, tok)
        return ner_dic
    
    @property
    def get_df(self):
        ner_dic = self.get_ner
        ner_dic['sms'] = ' '.join(self.sms[0])
        ner_dic['cls'] = self.get_cls
        df = pd.DataFrame.from_dict(ner_dic, orient='index')
        return df.T
    
    def _ner_res(self, sms, tokenizer, label2idx, model):
        '''sms type: 尚未预处理;
           且类型为：'[['Dear','Customer','you','have','sent','Rs.40.0']];
        '''
        new_sms_pro = preprocess(new_sms[0]).split(' ')
        token_seq = tokenizer.texts_to_sequences([new_sms_pro])
        x_te = pad_sequences(token_seq, padding = 'post', maxlen = maxlen)
        idx2word = {v: k for k, v in tokenizer.word_index.items()}
        show_word = [idx2word[i] if i!=0 else 'pad' for i in x_te[0]]
        p = model.predict(x_te)
        idx2label = {}
        for k, v in label2idx.items():
            idx2label[v] = k
        p = np.argmax(p, axis =- 1)
        return [idx2label[i] for i in p[0]], new_sms_pro

    def _ner_dic_form(self, labels, tokens):
        dic = {}
        dic['amount'] = ''.join([word for word, lab in zip(tokens, labels) if lab == 'I-amount'])
        dic['time'] = ''.join([word for word, lab in zip(tokens, labels) if lab == 'I-date'])
        dic['balance'] = ''.join([word for word, lab in zip(tokens, labels) if lab == 'I-bal'])
        return dic
    
    
    def _get_test_sample(self):
        import random
        sms = sample_template(sampled_template_path, banklist, num_samples=30)
        leng = len(sms)
        idx = random.sample(range(leng), 1)[0]
        sampled = sms.iloc[idx,:]
        sample_sms, sample_info = [sampled.sms.split()], sampled.bank + '_' + str(sampled.cluster_id)
        return sample_sms, sample_info

    def show_complete_res(self):
        new_sms, info = self._get_test_sample()
        print(new_sms)
        new_sms_pro = preprocess(new_sms[0]).split(' ')
        print(new_sms_pro)
        print(info)
        col = sampled_template_df.bank + '_' + sampled_template_df.cluster_id.astype(str)
        print('sample in banklist:{}'.format(info in set(col)))
        print('sample in training data:{}'.format(info.lower() in set(top10_data.sent_id)))
        token_seq = tokenizer.texts_to_sequences([new_sms_pro])
        x_te = pad_sequences(token_seq, padding = 'post', maxlen = maxlen)
        idx2word = {v: k for k, v in tokenizer.word_index.items()}
        show_word = [idx2word[i] if i!=0 else 'pad' for i in x_te[0]]

        p = model.predict(x_te)
        print('p len:{}'.format(len(p[0])))
        print('sms len:{}'.format(len(new_sms[0])))
        print('show word len:{}'.format(len(show_word)))
        idx2label = {}
        for k, v in label2idx.items():
            idx2label[v] = k
        prob = np.max(p, axis=-1)
        p = np.argmax(p, axis =- 1)

        print('{:15}||{:7}||{:7}||{:7}'.format('word', 'token', 'pred', 'prob'))
        print(30 * '=')
        for pred, show, word, prob in zip(p[0], show_word, new_sms_pro, prob[0]):
            print('{:15}||{:7}||{:7}||{:7}'.format(word, show, idx2label[pred], prob))

In [121]:
SFR = showFinalResult(new_sms, model, tokenizer, label2idx, top_10_sampled_template_df)

SFR.show_complete_res()

In [134]:
SFR.get_df

processing sentence num:0
 dear customer rs. 20. 00 has been charged for the transaction at west moti bagh sadar bazar dlin on 25-8-2018 as the monthly limit of free transactions is exceeded. updated balance: 1538. 08.

元组第三个代表模版ID:(0.5964518984275804, '流水-转出', 'PAYTMB_20')


,time,sms,cls,amount,balance
0,motibaghsadar,Dear Customer Rs.20.00 has been charged for th...,流水-转出,20.00,1538.08.


In [117]:
a

,time,sms,cls,amount,balance
0,motibaghsadar,Dear Customer Rs.20.00 has been charged for th...,流水-转出,20.00,1538.08.


In [51]:
new_sms, info = get_test_sample()
print(new_sms)
new_sms_pro = preprocess(new_sms[0]).split(' ')
print(new_sms_pro)
print(info)
col = sampled_template_df.bank + '_' + sampled_template_df.cluster_id.astype(str)
print('sample in banklist:{}'.format(info in set(col)))
print('sample in training data:{}'.format(info.lower() in set(top10_data.sent_id)))
token_seq = tokenizer.texts_to_sequences([new_sms_pro])
x_te = pad_sequences(token_seq, padding = 'post', maxlen = maxlen)
idx2word = {v: k for k, v in tokenizer.word_index.items()}
show_word = [idx2word[i] if i!=0 else 'pad' for i in x_te[0]]

p = model.predict(x_te)
print('p len:{}'.format(len(p[0])))
print('sms len:{}'.format(len(new_sms[0])))
print('show word len:{}'.format(len(show_word)))
idx2label = {}
for k, v in label2idx.items():
    idx2label[v] = k
prob = np.max(p, axis=-1)
p = np.argmax(p, axis =- 1)

print('{:15}||{:7}||{:7}||{:7}'.format('word', 'token', 'pred', 'prob'))
print(30 * '=')
for pred, show, word, prob in zip(p[0], show_word, new_sms_pro, prob[0]):
    print('{:15}||{:7}||{:7}||{:7}'.format(word, show, idx2label[pred], prob))

total templates among all the banks:1260
[['Dear', 'Customer', 'Rs.20.00', 'has', 'been', 'charged', 'for', 'the', 'transaction', 'at', 'WEST', 'MOTI', 'BAGH', 'SADAR', 'BAZAR', 'DLIN', 'on', '25-8-2018', 'as', 'the', 'monthly', 'limit', 'of', 'free', 'transactions', 'is', 'exceeded.', 'Updated', 'balance:', '1538.08.']]
['dear', 'customer', 'rs.', '20.', '00', 'has', 'been', 'charged', 'for', 'the', 'transaction', 'at', 'west', 'moti', 'bagh', 'sadar', 'bazar', 'dlin', 'on', '25-8-2018', 'as', 'the', 'monthly', 'limit', 'of', 'free', 'transactions', 'is', 'exceeded.', 'updated', 'balance:', '1538.', '08.']
PAYTMB_20
sample in banklist:True
sample in training data:True
p len:51
sms len:30
show word len:51
word           ||token  ||pred   ||prob   
dear           ||dear   ||other  ||0.9999998807907104
customer       ||customer||other  ||0.9999995231628418
rs.            ||rs.    ||other  ||0.9999998807907104
20.            ||<UNK>  ||I-amount||0.9937607645988464
00             ||<UNK>  

In [19]:
new_sms, info = get_test_sample()
print(info)
col = sampled_template_df.bank + '_' + sampled_template_df.cluster_id.astype(str)
print('sample in banklist:{}'.format(info in set(col)))
print('sample in training data:{}'.format(info.lower() in set(top10_data.sent_id)))
p = predict(model=model, maxlen=maxlen, tokenizer=tokenizer, new_sms=new_sms, show=True, label2idx=label2idx)



total templates among all the banks:1260
CSHBCK_1
sample in banklist:True
sample in training data:False
[[1 2 2 2 2 0 0 2 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
token          ||pred   
Open                     ||I-amount
Zero                     ||other
Balance                  ||other
A/c                      ||other
with                     ||other
Kotak                    ||I-date
Mahindra                 ||I-date
Bank                     ||other
Click                    ||I-bal
http://bit.ly/2x43TvjT&C*||I-bal


In [20]:
def show_train_res():
    sms = top10_data.token
    label = top10_data.lable